# musoW Twitter Pipeline

## Imports

In [1]:
path = '../'
import pandas as pd
#import custom functions
from PYTHON_FILES.LogReg_Searches import LogRegSearches

## Variables

In [2]:
# descriptions training set -> v2 = musow+mji descriptions vs summarized scrapes from twitter searches  
archive_desc_training_v2 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/archive_desc_training_v2.pkl')

# twitter training set -> v1 = tweets from bigrams vs tweets for digital humanities and music company 
twitter_training_set_v1 = pd.read_pickle(path+'LOGREG_RELEVANCE/TRAINING_SETS/twitter_training_v1.pkl')

## Training twitter and descriptions classifiers

This is a ONE TIME operation. The models are pickled and loaded later to predict new results from LOGREG_RELEVANCE/MODELS folder. 

In [ ]:
# one time training on twitter
twitter_training_model = LogRegSearches.train(twitter_training_set_v1, 'tweet', 'Target', 10, 'precision', 1000, 'twitter_pipeline', path)

# one time training on resources
resource_training_model = LogRegSearches.train(archive_desc_training_v2, 'Description', 'Target', 10, 'precision', 1000, 'resources_pipeline',path)

## Query Twitter

Calls Twitter API with a list of keywords and return results as raw csv and clean pickle in TWITTER_SEARCHES/RAW_SEARCHES folder.

In [ ]:
#load token
token = 'AAAAAAAAAAAAAAAAAAAAAJgsNAEAAAAAQcsgbUnOJJmqmU483%2F8x6n9V1i8%3Df0qaEo9cV1sWP4eyNQ6E9s8BiRjvFTSN9mSqithe8uIXSNP68x'
#a selection of keywords based on MJI and musoW datasets

#Choose keywords  
keywords = ['music archive', 'music collection']

#search timeframe (if using custom search)
start = ['2022-05-01T00:00:00.000Z', '2022-05-02T00:00:00.000Z', '2022-05-03T00:00:00.000Z', '2022-05-04T00:00:00.000Z', '2022-05-05T00:00:00.000Z', '2022-05-06T00:00:00.000Z', '2022-05-07T00:00:00.000Z']
end = ['2022-05-01T23:59:59.000Z', '2022-05-02T23:59:59.000Z', '2022-05-03T23:59:59.000Z', '2022-05-04T23:59:59.000Z', '2022-05-05T23:59:59.000Z', '2022-05-06T23:59:59.000Z', '2022-05-07T23:59:59.000Z']

#choose search option 
## search last week
tweets = LogRegSearches.search_twitter_weekly(token, keywords, 50, 50)
## search custom timeframe
#LogRegSearches.search_twitter_custom(token, better_keywords, start, end, 500, 500)

## Classify tweets

In [ ]:
#load all search results into a single dataframe 
tweets_to_classify = LogRegSearches.tweets_to_classify(path+'TWITTER_SEARCHES/RAW_SEARCHES/', f'{tweets[0][-16:]}.pkl')
tweets_to_classify

In [ ]:
#run classification and get links from results
predicted_tweets, twitter_link_list = LogRegSearches.predict_twitter(path, 'twitter_pipeline', tweets_to_classify, 'tweet', 1)
predicted_tweets

## Scrape URLS

In [ ]:
#scrape URL list and return a DF for resource classification
scraped_links = LogRegSearches.scrape_links(twitter_link_list, predicted_tweets, f'{tweets[0][-16:]}_scrapes')
scraped_links

## Classify web resources

In [ ]:
predicted_resources = LogRegSearches.predict_resource(path, 'resources_pipeline', scraped_links, 'Description', 1, f'{tweets[0][-16:]}')
predicted_resources